In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
df_data = pd.read_excel('dataset primer/ET_HARIAN.xlsx')

In [3]:
sisi_media_tanaman = {
    'A1' : 0.125, 'A2' : 0.125, 'A3' : 0.125, 'A4' : 0.125,
    'B1' : 0.150, 'B2' : 0.150, 'B3' : 0.150, 'B4' : 0.150,
    'C1' : 0.175, 'C2' : 0.175, 'C3' : 0.175, 'C4' : 0.175,
    'D1' : 0.200, 'D2' : 0.200, 'D3' : 0.200, 'D4' : 0.200,
    'E1' : 0.225, 'E2' : 0.225, 'E3' : 0.225, 'E4' : 0.225,
    'F1' : 0.250, 'F2' : 0.250, 'F3' : 0.250, 'F4' : 0.250,}

new_column_names = ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 
                    'C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3', 'D4', 
                    'E1', 'E2', 'E3', 'E4', 'F1', 'F2', 'F3', 'F4']

In [4]:
# Fungsi hitung luas tanaman
def luas_media(kode):
    if kode in sisi_media_tanaman:
        luas = sisi_media_tanaman[kode] ** 2
        return luas

In [5]:
# Fungsi utama menghitung gravimetri
def gravimetri_primer(berat, irigasi, luas_media_tanaman, curah_hujan, rho_air=1000):

    perubahan_berat = []
    perubahan_et = []

    # Menghitung turunan biasa dari jam ke-2 hingga jam ke-9
    for i in range(1, len(berat)):
        perubahan_berat.append(berat[i] - berat[i-1])

    # Menghitung nilai laju evapotranspirasi
    for i in range(len(perubahan_berat)):
        perubahan_et.append(curah_hujan[i] + irigasi[i] - (perubahan_berat[i]  / (luas_media_tanaman * rho_air)))

    return(perubahan_et)


In [6]:
# Pola ReGex
# Definisikan pola regex untuk kolom berat dan supply air
berat_pattern = r'^berat_[A-F][1-4]$'
supply_air_pattern = r'^supply_air_[A-F][1-4]$'

# Filter kolom yang sesuai dengan pola regex
berat_columns = [col for col in df_data.columns if re.match(berat_pattern, col)]
supply_air_columns = [col for col in df_data.columns if re.match(supply_air_pattern, col)]


In [7]:
# Hitung rata-rata dan jumlah untuk setiap kode
ET_harian = []

for kode in sisi_media_tanaman.keys():
    kode_berat = f'berat_{kode}'
    kode_irigasi = f'supply_air_{kode}'

    berat = df_data[kode_berat]
    irigasi = df_data[kode_irigasi].fillna(0).array
    curah_hujan = df_data['curah_hujan'].array
    # print(curah_hujan)

    if kode in sisi_media_tanaman:
        luas = sisi_media_tanaman[kode] ** 2
    # print(luas)

    evapotranspirasi = gravimetri_primer(berat, irigasi, luas, curah_hujan)
    # print(f"ET: {evapotranspirasi}")
    
    ET_harian.append(evapotranspirasi)

df_ET = pd.DataFrame(ET_harian)
df_ET_transposed = df_ET.T
df_ET_transposed.columns = new_column_names
df_ET_transposed.to_csv('hasil/Hasil_ET_Harian.csv', index=True)

df_ET_transposed


,A1,A2,A3,A4,B1,B2,B3,B4,C1,C2,...,D3,D4,E1,E2,E3,E4,F1,F2,F3,F4
0,0.495904,0.490976,0.491936,0.493472,NaN,NaN,NaN,NaN,1.238767,1.234882,...,1.242825,1.229400,2.468988,2.464563,2.467763,2.494390,2.471008,2.471648,2.475056,2.472528
1,0.499232,0.498912,0.501024,0.724784,NaN,NaN,NaN,NaN,1.247649,1.245624,...,1.141375,1.140500,2.493659,2.487970,2.491052,2.492770,2.486880,2.640432,2.490880,2.483280
2,0.998848,0.998464,1.000960,1.022336,NaN,NaN,NaN,NaN,1.747159,1.749020,...,1.746125,1.745350,2.497768,2.495615,2.496622,2.497610,2.502624,2.339920,2.494976,2.496704
3,0.509072,0.508368,0.510864,0.508816,NaN,NaN,NaN,NaN,1.009722,1.006490,...,1.007000,1.003875,1.506928,1.510899,1.512341,1.509378,1.508384,1.510816,1.510464,1.503168
